In [1]:
import cv2
import numpy as np
import matplotlib as plt
import pathlib
from os import listdir
from os.path import isfile, join

In [2]:
# define location of dataset and return all files
dataset_location = "C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output"
target_dir = "C:/Users/Legos/Documents/PhD/FARTS/generated_data/YOLO_latest_3D_synth_test"
all_files = [f for f in listdir(dataset_location) if isfile(join(dataset_location, f))]

# next, sort files into images, depth maps, segmentation maps, data, and colony info
dataset_img = []
dataset_depth = []
dataset_seg = []
dataset_data = []
dataset_colony = dataset_location + "/ColonieInfo.csv"

for file in all_files:
    loc = dataset_location + "/" + file
    if file[-7:-4] == "Img":
        dataset_img.append(loc)
    elif file[-7:-4] == "Seg":
        dataset_seg.append(loc)
    elif file[-8:-4] == "Depth":
        dataset_depth.append(loc)
    elif file[-8:-4] == "Data":
        dataset_data.append(loc)
        
print("Found",len(all_files),"files...")

# next sort the colony info into its IDs to determine the colony size and individual scales
# one entry for each successive ID is read
from csv import reader

colony = {'seed': 0,
            'ID': [],
         'scale': [],
        'weight': []}

with open(dataset_colony, 'r') as colony_file:
        print("reading", file)
        # pass the file object to reader() to get the reader object
        csv_reader = reader(colony_file)
        # iterate over each row in the csv using reader object
        for r, row in enumerate(csv_reader):
            if r == 0:
                colony['seed'] = row[0].split("=")[-1]
            else:
                colony['ID'].append(row[0].split("=")[-1])
                colony['weight'].append(row[1].split("_")[1])
                colony['scale'].append(float(row[2].split("=")[-1]))

print("Loaded colony file with seed",colony['seed'],"and",len(colony['ID']),"individuals.")

# get provided classes to create a dictionary of class IDs and class names
subject_class_names = np.unique(np.array(colony["weight"]))
subject_classes = {}
for id,sbj in enumerate(subject_class_names):
    subject_classes[str(sbj)] = id

print("\nA total of",len(subject_class_names),"unique classes have been found.")
print("The classes and respective class IDs are:\n",subject_classes)

Found 55 files...
reading ColonieInfo.csv
Loaded colony file with seed  14256 and 100 individuals.

A total of 4 unique classes have been found.
The classes and respective class IDs are:
 {'00011': 0, '00262': 1, '00501': 2, 'Sungaya': 3}


Now that we have the cleaned colony info, we can start loading the data associated with each frame.
For simplicity we will simply this a list of list as the number of individuals.

We will therefore access "data" as [frame] [individual] [attribute], where attributes will include [ID,bbox_x_0,bbox_y_0,...]

for now training and evaluating detectors, only the bounding box (and ID) info will be relevant

In [3]:
data = []

for file in dataset_data:
    # store all returned coordinates for each individual
    coords = []
        
    # open file in read mode
    with open(file, 'r') as read_obj:
        print("reading", file)
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # iterate over each row in the csv using reader object
        for row in csv_reader:
            # exclude camera projection row
            if not row[0].split(".")[0] == "camera_projection:":
                individual = [float(row[0].split(".")[0])]
                # row variable is a list that represents a row in csv
                for elem in row:
                    try:
                        individual.append(float(elem.split("=")[-1]))
                    except ValueError:
                        pass
                coords.append(individual)
        
    data.append(coords)
    
print("\nThe dataset has a total of", len(data),"generated frames.")

reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/10_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/11_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/12_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/13_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/14_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/15_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/16_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/17_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/18_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/1_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/2_Data.csv
reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/3_Data.csv
reading C:/Users/Le

As there may be animals for which we don't use all bones we can return a list of all labels and exclude the respective locations from the pose data. As all animals use the same convention, we can simply read in one example and remove the corresponding indices from all animals.

In [4]:
"""
# first open and read the first line from the first imported data file
labels = []
entries_found = False
entry = 0

while not entries_found:
    with open(dataset_data[entry], 'r') as read_obj:
        print("reading", read_obj.name)
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        row_0 = next(csv_reader)  # gets the first line
        # iterate over each row in the csv using reader object
        if row_0[0][:3] != "cam":
            entries_found = True
            for elem in row_0:
                try:
                    labels.append((elem.split("=")[0].split("Bone.")[-1]))
                except ValueError:
                    pass
        else:
            print("No entries found! Reading next file... \n")
            entry += 1

# show all used labels:
print("\nAll labels:")
print(labels)
"""

# first open and read the first line from the first imported data file
labels = []
with open(dataset_data[0], 'r') as read_obj:
    print("reading", file)
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    row_0 = next(csv_reader)  # gets the first line
    # iterate over each row in the csv using reader object
    for elem in row_0:
        try:
            labels.append((elem.split("=")[0].split("Bone.")[-1]))
        except ValueError:
            pass

# now let's define which labels NOT to use (in our case, all labels relating to wings)
# ... so that just means "omit all lables that start with 'w'"

matched_labels = []# [i for i, item in enumerate(labels) if item in omit_labels]
for l, label in enumerate(labels):
    if label[0] == "w":
        matched_labels.append(l-1)
        
print("\nCorresponding to the following indices:",matched_labels)

# time to remove them from all individuals in "data"
for f, frame in enumerate(data):
    for i, individual in enumerate(frame):
        ind_temp = np.array(data[f][i])
        data[f][i] = np.delete(ind_temp, matched_labels)


# show all used labels:
print("\nAll labels:")
print(labels)

reading C:/Users/Legos/Documents/PhD/FARTS/UNREAL/FARTS_STICKS/Output/9_Data.csv

Corresponding to the following indices: [313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352]

All labels:
['5.BoundingBox.BoundMin.X', 'BoundingBox.BoundMin.Y', 'BoundingBox.BoundMax.X', 'BoundingBox.BoundMax.Y', 'b_t.X', 'b_t.Y', 'b_t.X_world', 'b_t.Y_world', 'b_t.Z_world', 'b_a_1.X', 'b_a_1.Y', 'b_a_1.X_world', 'b_a_1.Y_world', 'b_a_1.Z_world', 'b_a_2.X', 'b_a_2.Y', 'b_a_2.X_world', 'b_a_2.Y_world', 'b_a_2.Z_world', 'b_a_3.X', 'b_a_3.Y', 'b_a_3.X_world', 'b_a_3.Y_world', 'b_a_3.Z_world', 'b_a_4.X', 'b_a_4.Y', 'b_a_4.X_world', 'b_a_4.Y_world', 'b_a_4.Z_world', 'b_a_5.X', 'b_a_5.Y', 'b_a_5.X_world', 'b_a_5.Y_world', 'b_a_5.Z_world', 'b_a_5_end.X', 'b_a_5_end.Y', 'b_a_5_end.X_world', 'b_a_5_end.Y_world', 'b_a_5_end.Z_world', 'l_1_co_r.X', 'l_1_co_r.Y', 'l_1_co_

Now that we have loaded data and colony info we can start plotting bounding boxes on top of their respective images

In [5]:
# transform between sRGB and linear colour space (optional)

def to_linear(srgb):
    linear = np.float32(srgb) / 255.0
    less = linear <= 0.04045
    linear[less] = linear[less] / 12.92
    linear[~less] = np.power((linear[~less] + 0.055) / 1.055, 2.4)
    return linear * 255.0

    
def from_linear(linear):
    srgb = linear.copy()
    less = linear <= 0.0031308
    srgb[less] = linear[less] * 12.92
    srgb[~less] = 1.055 * np.power(linear[~less], 1.0 / 2.4) - 0.055
    return srgb * 255.0

In [6]:
# create unique colours for each ID
import numpy as np
import time

# alright. Let's take it from the top and fucking multi-thread this.
import threading
import queue
import sys
import os

def getThreads():
    """ Returns the number of available threads on a posix/win based system """
    if sys.platform == 'win32':
        return int(os.environ['NUMBER_OF_PROCESSORS'])
    else:
        return int(os.popen('grep -c cores /proc/cpuinfo').read())

class myThread(threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q

    def run(self):
        print("Starting " + self.name)
        process_detections(self.name, self.q)
        print("Exiting " + self.name)
        
def createThreadList(num_threads):
    threadNames = []
    for t in range(num_threads):
        threadNames.append("Thread_" + str(t))

    return threadNames

def process_detections(threadName, q):
    while not exitFlag_stacking:
        queueLock.acquire()
        if not workQueue_stacking.empty():
            
            data_input = q.get()
            i = data_input[0]
            print(i)
            img = data_input[1]
            queueLock.release()
            
            display_img = cv2.imread(img)
            display_img_out = display_img.copy()

            if generate_dataset:
                img_info = []
                
            # compute visibility for each individual
            seg_img = cv2.imread(dataset_seg[i])
            
            individual_visible = False

            for im, individual in enumerate(data[i]):

                fontColor = (int(ID_colours[int(individual[0]),0]),
                             int(ID_colours[int(individual[0]),1]),
                             int(ID_colours[int(individual[0]),2]))
                if enforce_tight_bboxes:
                    bbox = fix_bounding_boxes(individual[1:],max_val=display_img.shape)
                else:
                    bbox = fix_bounding_boxes(individual[1:5],max_val=display_img.shape)

                # FOR SOME REASON, OCCASIONALLY, THE ID OF THE SEG FILE IS LOWER THAN THE DATA FILE
                # with: ID = red_channel/255 * 50
                # red_channel = (ID/50) * 255
                ID_red_val = int((individual[0]/len(colony['ID']))*255)
                try:
                    ID_mask = cv2.inRange(seg_img[bbox[1]:bbox[3],bbox[0]:bbox[2]], np.array([0,0, ID_red_val - 5]), np.array([0,0, ID_red_val + 5]))
                    indivual_occupancy = cv2.countNonZero(ID_mask)
                except:
                    if len(threadList_stacking) == 1: 
                        print("Individual fully occluded:",im,"in",dataset_seg[i])
                    indivual_occupancy = 1
                
                #indivual_occupancy = np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 - 1)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 + 1)]).all(axis = 2))
                bbox_area = abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1])) + 1
                bbox_occupancy = indivual_occupancy / bbox_area
                #print("Individual", individual[0], "with bounding box occupancy ",bbox_occupancy)

                cv2.putText(display_img, "ID: " + str(int(individual[0])), (bbox[0] + 10,bbox[3] - 10), font, fontScale, fontColor, lineType)
                class_ID = subject_classes[colony['weight'][int(individual[0])]] # here we use a single class, otherwise this can be replaced by size / scale values
                        
                if bbox_occupancy > visibility_threshold:
                    
                    individual_visible = True
                            
                    if generate_dataset:
                        # now we need to convert the bounding box info into the desired format.
                        img_dim = display_img.shape

                        # [class_ID, centre_x, centre_y, bounding_box_width, bounding_box_height]
                        valid_new_x = False
                        valid_new_y = False
                        
                        if enforce_centred_bboxes:
                            # coords of head
                            b_t = np.array([individual[5], individual[6]])
                            # coords of abdomen
                            b_a_1 = np.array([individual[10], individual[11]])
                            # compute new centre point
                            new_centre_x = (individual[5] + individual[10]) / 2
                            new_centre_y = (individual[6] + individual[11]) / 2
                            
                            if new_centre_x < img_dim[1] and new_centre_x > 0:
                                centre_x = new_centre_x / img_dim[1]
                                valid_new_x = True
                            
                            if new_centre_y < img_dim[0] and new_centre_y > 0:
                                centre_y = new_centre_y / img_dim[0]
                                valid_new_y = True
                            
                            cv2.circle(display_img, (int(new_centre_x),int(new_centre_y)), 
                                       radius=3, color=fontColor, thickness=-1)    
                        
                        for label in range(int((len(individual)-5)/5)):
                            cv2.circle(display_img, (int(individual[label*5+5]),
                                                     int(individual[label*5+6])), 
                                       radius=3, color=fontColor, thickness=-1)    
    
                        
                        bounding_box_width = abs(bbox[2] - bbox[0]) / img_dim[1]
                        bounding_box_height = abs(bbox[3] - bbox[1]) / img_dim[0]
                        
                        if not valid_new_x or not valid_new_y:
                            centre_x = bbox[0] / img_dim[1] + bounding_box_width / 2
                            centre_y = bbox[1] / img_dim[0] + bounding_box_height / 2
                            
                        img_info.append([class_ID,centre_x,centre_y,bounding_box_width,bounding_box_height])
                        
                        cv2.rectangle(display_img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), fontColor, 2)
                        
                else:
                    pass
                    if len(threadList_stacking) == 1: 
                        print("Ah shit, can't see",int(individual[0]),class_ID)

                if generate_dataset and individual_visible:
                    
                    img_name = target_dir + "/obj/" + img.split('/')[-1][:-4] + "_synth" + ".JPG"
                    cv2.imwrite(img_name, display_img_out)
                    
                    with open(target_dir + "/obj/" + img.split('/')[-1][:-4] + "_synth" + ".txt", "w") as f: 
                        output_txt = []
                        if img_info:
                            for line in img_info:
                                line_str = ' '.join([str(i) for i in line])
                                output_txt.append(line_str+"\n")
                            f.writelines(output_txt)
                        else:
                            f.write("")
                            
            if len(threadList_stacking) == 1:
                cv2.imshow("labeled image", cv2.resize(display_img, (int(display_img.shape[1] / 2), 
                                                                     int(display_img.shape[0] / 2))))
                cv2.waitKey(0)

        else:
            queueLock.release()
            
            
####################################################################################################
            
# set True to show processing results for each image (disables parallel processing)
DEBUG = False

# we can additionally plot the points in the data files to check joint locations
# WARNING: At the moment there seems to be an issue with inccorrectly given joint locations
plot_joints = False

# remember to refine an export folder when saving out your dataset
generate_dataset = True

# we can enforce the bounding box to centre on the individual instead of being influenced by its orientation
# As the groundtruth in real recordings is annotated in the same way this should boost the average accuracy
enforce_centred_bboxes = False

# alternatively, we can draw tighter bounding boxes without enforced centres, based on 2D keypoints
enforce_tight_bboxes = True #one or the other. This option will overwrite "enforce_centred" if True

####################################################################################################
            
# setup as many threads as there are (virtual) CPUs
exitFlag_stacking = 0
# only use a fourth of the number of CPUs for stacking as hugin and enfuse utilise multi core processing in part
if DEBUG:
    threadList_stacking = createThreadList(1)
else:
    threadList_stacking = createThreadList(getThreads())
print("Using", len(threadList_stacking), "threads to parse data...")
queueLock = threading.Lock()

# define paths to all images and set the maximum number of items in the queue equivalent to the number of images
workQueue_stacking = queue.Queue(len(dataset_img))
threads = []
threadID = 1


np.random.seed(seed=1)
ID_colours = np.random.randint(255, size=(len(colony['ID']), 3))

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
lineType = 2

def fix_bounding_boxes(coords,max_val=[1024,1024],exclude_wings=True):
    # fix bounding box coordinates so they do not reach beyond the image
    # you can either pass only bounding box coordinates or the entire skeleton coordinates
    # The latter will recalculate a tighter bounding box, based on all keypoints
    # When recalculating the bounding box based on all keypoints, you can chose to ignore wings.
    fixed_coords = []
    
    if len(coords) == 4:
        coords_bbox = coords[:4]
    
    else:
        coords_bbox = [0,0,max_val[0],max_val[1]]
        # get all X and Y coordinates to find min and max values for the bounding box
        key_x = coords[4::5]
        key_y = coords[5::5]
        
        coords_bbox[0] = max([0,min(key_x)])
        coords_bbox[1] = max([0,min(key_y)])
        coords_bbox[2] = min([max_val[0],max(key_x)])
        coords_bbox[3] = min([max_val[1],max(key_y)])
    
    for c, coord in enumerate(coords_bbox):
        if c == 0 or c == 2:
            max_val_temp = max_val[1]
        else:
            max_val_temp = max_val[0]
            
        if coord >= max_val_temp:
            coord = max_val_temp
        elif coord <= 0:
            coord = 0
        
        fixed_coords.append(int(coord))
        
    return fixed_coords

if generate_dataset:
    from helper.Generate_YOLO_training import createCustomFiles
    createCustomFiles(output_folder=target_dir+"/",obIDs=subject_class_names)

# determine the proportion of a bounding box that needs to be filled before considering the visibility as too low
# WARNING: At the moment the ID shown in segmentation maps does not always correspond to the ID in the data file (off by 1)
visibility_threshold = 0.015

timer = time.time()

# Create new threads
for tName in threadList_stacking:
    thread = myThread(threadID, tName, workQueue_stacking)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue with stacks
queueLock.acquire()
for i,img in enumerate(dataset_img):
    workQueue_stacking.put([i, img])
queueLock.release()

# Wait for queue to empty
while not workQueue_stacking.empty():
    pass

# Notify threads it's time to exit
exitFlag_stacking = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print("Exiting Main Stacking Thread")

# now run createCustomFiles again, to update the train.txt and test.txt files to include the paths to the respective images
if generate_dataset:
    createCustomFiles(output_folder=target_dir+"/")

# close all windows if they were opened
cv2.destroyAllWindows()

print("Total time elapsed:",time.time()-timer,"seconds")

Using 12 threads to parse data...
Using custom labels: ['00011' '00262' '00501' 'Sungaya']
Using 16 training images and 1 test images. (10.0 %)
Successfully created all required files!
Starting Thread_0
Starting Thread_1
Starting Thread_2
Starting Thread_3
Starting Thread_4
Starting Thread_5
Starting Thread_6
Starting Thread_7
Starting Thread_8
Starting Thread_9
Starting Thread_10
Starting Thread_11
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
Exiting Thread_11
Exiting Thread_8
Exiting Thread_7
Exiting Thread_6
Exiting Thread_3
Exiting Thread_2
Exiting Thread_5
Exiting Thread_4
Exiting Thread_9
Exiting Thread_1
Exiting Thread_10
Exiting Thread_0
Exiting Main Stacking Thread
Using 16 training images and 1 test images. (10.0 %)
Successfully created all required files!
Total time elapsed: 3.6305675506591797 seconds


Now, re-run **Generate_YOLO_training()** to combine all files into their final test/train sets

In [7]:
from helper.Generate_YOLO_training import createCustomFiles
createCustomFiles(output_folder=target_dir+"/",obIDs=subject_class_names)

Using custom labels: ['00011' '00262' '00501' 'Sungaya']
Using 16 training images and 1 test images. (10.0 %)
Successfully created all required files!


In [8]:
# command to train yolo based on generated data:
